# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [24]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [25]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

c:\Users\tew\Training\data_engineering_udacity\5_Data_Modeling_With_Apache_Cassandra


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [26]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [27]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


In [28]:
def create_df(colnames, data):
    return pd.DataFrame(rows, columns=colnames)

# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [29]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [30]:

try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS data_modeling 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [31]:
try:
    session.set_keyspace('data_modeling')
except Exception as e:
    print(e)


### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [ ]:
session.row_factory = create_df

In [ ]:
table_name = "music_library"


    
query = f"CREATE TABLE IF NOT EXISTS  {table_name}"
query = query + """(sessionid INT,
                    iteminsession INT,
                    artist TEXT,
                    song TEXT,
                    song_length FLOAT,
                    PRIMARY KEY (sessionId,itemInSession))
                    """
try:
    session.execute(query)
except Exception as e:
    print(e)

Error from server: code=2200 [Invalid query] message="Table 'data_modeling.music_library' doesn't exist"


In [ ]:
file = 'event_datafile_new.csv'
df = pd.read_csv(file)
column_names = df.columns.tolist()
column_names

['artist',
 'firstName',
 'gender',
 'itemInSession',
 'lastName',
 'length',
 'level',
 'location',
 'sessionId',
 'song',
 'userId']

In [ ]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#


with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO music_library (sessionid, iteminsession, artist, song, song_length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        columns_to_values = dict(zip(column_names,line))
        session.execute(query, (
                                int(columns_to_values['sessionId']),
                                int(columns_to_values['itemInSession']),
                                columns_to_values['artist'],
                                columns_to_values['song'],
                                float(columns_to_values['length']))
                        )

#### Do a SELECT to verify that the data have been inserted into each table

In [ ]:
query = "select artist, song, song_length  from music_library WHERE sessionid=338 and iteminsession=4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row)

artist
song
song_length


In [ ]:
# expected output for the query 1
cond = (df['sessionId']==338) & (df['itemInSession']==4)
df[cond]

In [ ]:
# Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4
query1 = "select artist, song, song_length  from music_library WHERE sessionid=338 and itemInSession=4"
try:
    results = session.execute(query1)
except Exception as e:
    print(e)
    
df_query2 = results._current_rows 
df_query2

,artist,song,song_length


In [ ]:
for row in results:
    print (row)

artist
song
song_length


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [ ]:
## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

table_name = "users_songs"

query = f"drop table {table_name}"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = f"CREATE TABLE IF NOT EXISTS {table_name} "
query = query + """(sessionid INT,
                    iteminsession INT,
                    userid INT,   
                    artist TEXT,
                    song TEXT,
                    first_name TEXT,
                    last_name TEXT,
                    PRIMARY KEY ((userid, sessionid), iteminsession))
                    """
try:
    session.execute(query)
except Exception as e:
    print(e)

                    

Error from server: code=2200 [Invalid query] message="Table 'data_modeling.users_songs' doesn't exist"


In [40]:
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for i, line in enumerate(csvreader):
        query = "INSERT INTO users_songs (sessionid, iteminsession, userid, artist, song, first_name, last_name)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        columns_to_values = dict(zip(column_names,line))
        session.execute(query, (
                                int(columns_to_values['sessionId']),
                                int(columns_to_values['itemInSession']),
                                int(columns_to_values['userId']),
                                columns_to_values['artist'],
                                columns_to_values['song'],
                                columns_to_values['firstName'],
                                columns_to_values['lastName']
                                )
                        )


In [41]:
# expected output 
cond = (df['userId']==10) & (df['sessionId']==182)
df[cond]

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
151,Down To The Bone,Sylvie,F,0,Cruz,333.76608,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",182,Keep On Keepin' On,10
152,Three Drives,Sylvie,F,1,Cruz,411.63710,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",182,Greece 2000,10
153,Sebastien Tellier,Sylvie,F,2,Cruz,377.73016,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",182,Kilometer,10
154,Lonnie Gordon,Sylvie,F,3,Cruz,181.21098,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",182,Catch You Baby (Steve Pitron & Max Sanna Radio...,10


In [42]:
##  Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
query2 = "select artist, song, first_name, last_name from users_songs WHERE userid=10 and sessionid=182"
try:
    results =  session.execute(query2)
except Exception as e:
    print(e)

df_query2 = results._current_rows 
df_query2

,artist,song,first_name,last_name


In [43]:
for row in results:
    print (row)


artist
song
first_name
last_name


In [44]:
results


In [45]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

query = "CREATE TABLE IF NOT EXISTS users_info "
query = query + """(userid INT,
                    song TEXT,
                    first_name TEXT,
                    last_name TEXT,
                    PRIMARY KEY (song,first_name, last_name, userid))
                    """
try:
    session.execute(query)
except Exception as e:
    print(e)
                    

In [46]:
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for i, line in enumerate(csvreader):
        query = "INSERT INTO users_info (userid, song, first_name, last_name)"
        query = query + " VALUES (%s, %s, %s, %s)"
        columns_to_values = dict(zip(column_names,line))
        session.execute(query, (
                                int(columns_to_values['userId']),
                                columns_to_values['song'],
                                columns_to_values['firstName'],
                                columns_to_values['lastName']
                                )
                        )

In [ ]:
# expected output for the query 3
cond = (df['song']=='All Hands Against His Own'
df[cond]

In [ ]:
# Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
query3 = "select userid, song, first_name, last_name  from users_info WHERE song='All Hands Against His Own'"
try:
    results = session.execute(query3)
except Exception as e:
    print(e)
    
df_query2 = results._current_rows 
df_query2

,userid,song,first_name,last_name


### Drop the tables before closing out the sessions

In [ ]:
## TO-DO: Drop the table before closing out the sessions

In [ ]:
table_names = ["users_songs", "users_info", "music_library"]

for table_name in table_names:
    query = f"drop table {table_name}"
    try:
        rows = session.execute(query)
    except Exception as e:
        print(e)



### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()